In [1]:
# import all necessary libraries
!pip install mysql-connector-python
import pandas as pd
import numpy as np
import sklearn
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import time
import json
import pickle
from datetime import datetime
import requests
import mysql.connector
from mysql.connector import errorcode
%matplotlib inline
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [2]:
# define a mysql connection to aws db
def aws_connect(db_name):
    connection = mysql.connector.connect(
    host = config.AWS_ENDPOINT,
    user = config.AWS_USER,
    passwd = config.AWS_PASSWORD,
    port = config.AWS_PORT,
    database = db_name)
    return connection

# create a connection with aws db
cnx = aws_connect('FAANG_raw')

In [3]:
# create a cursor over the mysql connection
cursor = cnx.cursor()

In [4]:
# function to create our AWS database
def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [5]:
# name our database
db_name = 'FAANG_raw'

# check to see if the database already exists, and if it doesn't, create it
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [6]:
# template
TABLES = {}
TABLES['tweets1'] = (
    "CREATE TABLE tweets1 ("
    "  tweet varchar(250) NOT NULL,"
    "  datetime date NOT NULL,"
    "  username varchar(50) NOT NULL"
    ") ENGINE=InnoDB")

In [7]:
# create tables according to above template
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

Creating table tweets1: already exists.


In [16]:
# create a mysql connection
cnx = aws_connect('FAANG_raw')

# create a cursor over the mysql connection
cursor = cnx.cursor()

# insert statement to add a tweet
add_tweet = ("INSERT INTO tweets1"
               "(tweet, datetime, username)"
               "VALUES (%s, %s, %s)")

# listener class definition
class listener(StreamListener):

    def on_data(self, data):
        all_data = json.loads(data)
        tweet = all_data["text"]
        username = all_data["user"]["screen_name"]
        cursor.execute(add_tweet, (tweet, time.time(), username))
        cnx.commit()
        print(tweet)
        return True

    def on_error(self, status):
        print(status)
        return False

In [17]:
# twitter api access info
ckey = config.TWITTER_API_KEY
csecret = config.TWITTER_API_SECRET_KEY
atoken = config.TWITTER_ACCESS_TOKEN
asecret = config.TWITTER_ACCESS_TOKEN_SECRET

In [18]:
# stream filter
auth = OAuthHandler(ckey, csecret)
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=["$FB", "$AMZN", "$APPL", "$NFLX", "$GOOGL"])

Mediaset CFO says ProSieben important partner but not the only one $NFLX $GOOGL https://t.co/UGcK6p9Jui
$NSPX 1 million on the bid! WHALES🐳💦💦 #BIOTECH #cancer💸  $spx $es_f #daytrading #investing #futures $spy $nasdaq… https://t.co/lDqinPDFEq
$FB https://t.co/midlhFi9Ez
RT @bespokeinvest: Covid Economy stocks flying today.  $TDOC $CHGG $CHWY $AMZN $ZM $WORK $ATVI $EA
$SPY $SPX $ES_F 2880 / 2900 region should present resistance once again but nice rally off lows of 2830s from the o… https://t.co/WoA9zCIpVV
Jumped into $NFLX @ 446.03
Join with our free chat room. https://t.co/ORJzEKuuEa

$TLRY $APHA $SPY $QQQ $DJIA $DIA #stockmarket #investing… https://t.co/ln4RMiWatE
$AMZN out half the short +10
$zm $tdoc two strong software on day, virus names
Market wasn't going to shake me out out this morning instead I added to every open position. That equated to some n… https://t.co/gBnITj2d3O
@realDonaldTrump $AMZN
$SPY $ES_F
$QQQ #NASDAQ https://t.co/OVxWRlehJD
3 Unstoppable Industries to Invest 

KeyboardInterrupt: 

In [20]:
auth.get_authorization_url()

'https://api.twitter.com/oauth/authorize?oauth_token=TWDeWAAAAAABDuaUAAABcg6PDuM'